In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import gc
from captum.attr import *
import quantus
from torch.utils.data import DataLoader
import gc
import torchvision.transforms as transforms
import torchvision.models as models

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# the validation transforms
valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [5]:
images = '/home/db1702/Downloads/imagenet-mini/train/'

In [6]:
def load_imagenet_model():
    model = models.resnet50(pretrained=True).to(device)
    model.to('cuda')
    model.train(False)
    return model

In [7]:
normal_model = load_imagenet_model()
normal_model.to(device)
normal_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
#get dataset
test = torchvision.datasets.ImageFolder(images, transform=valid_transform)
test_loader = DataLoader(test, shuffle=True, batch_size = 2)

# For given adversarial images and benign images, collect metrics of feature attribution sensitivity and model prediction sensitivity. Save in csv that will be used for inspecting detection performance. 

In [9]:
import pandas as pd

In [10]:
def make_noise(x_batch, y_batch, spread):
    new_x_batch = []
    for x in x_batch:
        x = x.data.cpu().numpy()
        stdev = spread * (np.max(x)-np.min(x))
        noise = np.random.normal(0, stdev, x.shape).astype(np.float32)
        x_plus_noise = x + noise
        x_plus_noise = np.clip(x_plus_noise, 0, 1)
        x_plus_noise = torch.from_numpy(x_plus_noise).cpu()
        new_x_batch.append(x_plus_noise)
    new_batch = torch.stack(new_x_batch).to(device)
    return new_batch

In [11]:
# Define uniform noise function
def add_uniform_noise(image):
    # Generate uniform noise with mean 0 and standard deviation 25
    noise = np.random.uniform(low=-0.5, high=0.5, size=image.shape).astype(np.float32)
    noisy_image = np.clip(image + noise, 0, 1).astype(np.uint8)
    return noisy_image

def uniform_noise(x_batch, y_batch): 
    # Convert batch of images to numpy array
    images = x_batch.detach().cpu().numpy().transpose(0, 2, 3, 1) * 1.0
    # Add Poisson noise to each image in the batch
    noisy_images = [add_uniform_noise(image) for image in images]
    # Convert noisy images back to Tensor format
    noisy_inputs = torch.from_numpy(np.array(noisy_images).transpose(0, 3, 1, 2) / 1.0).float()
    return noisy_inputs.to(device)


In [12]:
def compute_metrics_benign(adv_path, normal_model): 
    
    print("Computing metrics for {} for benign")
    
    npobj = np.load(adv_path)
    adaptive_image = npobj['b_images']
    adaptive_label = npobj['b_labels']
    
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
        attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
        logit_uniform
    ], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution",
        "uniform attr",
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
        "uniform logit"
                    ])
            
    path = "adaptive_Benign.csv"
    df.to_csv(path)

In [13]:
def compute_metrics_adv(adv_path, normal_model): 
    
    print("Computing metrics for {} for adv")
    npobj = np.load(adv_path)
    adaptive_image = npobj['a_images']
    adaptive_label = npobj['a_labels']
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    attribution_uniform = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    logit_uniform = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.25)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.35)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        uniform_noisy_images = uniform_noise(images_adv, y_pred_adv)
        uniform_logits = normal_model(uniform_noisy_images)
        
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        diff4 = torch.norm(x_logits-uniform_logits,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        logit_uniform.extend(diff4.detach().cpu().numpy())
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        a_batch_uniform = quantus.explain(
        model=normal_model, inputs=uniform_noisy_images, targets=y_pred_adv, **{"method:": "Saliency", "device": device})
        
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        for a, b in zip(a_batch, a_batch_uniform):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_uniform.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
        attribution_uniform,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3,
    logit_uniform], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution",
        "uniform attr",
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
        "uniform logit"
                    ])
            
    path = "adaptive_Adv.csv"
    df.to_csv(path)

In [14]:
def compute_TPR(adv1, a, b, adv2, c, d): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2 in zip(adv1, adv2): 
        if value1<a or value1>b:
            TP += 1
        else:
            if value2<c or value2>d:
                TP+=1
            else: 
                FN+=1
    
    return (TP/(TP+FN))*100

In [15]:
def compute_FPR(ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6, value7 in zip(ap2a,ap2b):
        if value6<k or value6>l:
            FP +=1
        else:
            if value7<m or value7>n:
                FP +=1

    return (FP/(len(ap2a)))*100

In [16]:
import sklearn
from sklearn.metrics import roc_auc_score

In [17]:
def return_auc(adv_path, model):
    #logitgaussian3
#     k=[1810,2510,2810,3000, 3200,3500, 6000]
#     l=[6700,6700,6700,6700,6700,6700, 6700]
    
    k=[1023, 1300,1490,1580, 1600, 1700, 1800, 1900, 2100]
    l=[2970, 2970,2970,2970, 2970, 2970, 2970, 2970, 2970]
    
    m=[1340, 1900, 2200, 2400,  2800, 3000, 3400, 5000,7000 ]
    n=[10894, 10894, 10894,10894,10894,10894, 10894, 10894, 10894]

    
    #attrgaussian3
#     m=[1400,1750,1900, 2300, 3200, 3500, 6000]
#     n=[6600,6600,6600,6600,6600,6600,6600]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[6].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t1,t2,t3,t4 in zip(k,l,m,n):
        FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[6].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t1,t2,t3,t4 in zip(k,l,m,n):
        TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), tpr_results, fpr_results)

# FGSM

In [18]:

adv_path1 = 'adv samples/IMAGENET/ResNet50/FGSM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/FGSM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/FGSM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/FGSM/0.25098039215686274eps.npz'

In [19]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.6406354999999999,
 [0.01, 0.099, 0.27, 0.412, 0.53, 0.684, 0.84, 0.985, 1.0],
 [0.005, 0.044, 0.136, 0.22, 0.327, 0.462, 0.667, 0.932, 0.995])

In [20]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.7574264999999998, [0.01, 0.135, 0.387, 0.571, 0.72, 0.855, 0.949, 0.9990000000000001, 1.0], [0.0, 0.034, 0.128, 0.21600000000000003, 0.317, 0.447, 0.653, 0.925, 0.995])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.8570720000000002, [0.015, 0.287, 0.607, 0.7760000000000001, 0.925, 0.971, 0.9990000000000001, 1.0, 1.0], [0.002, 0.044, 0.126, 0.23799999999999996, 0.336, 0.466, 0.654, 0.936, 0.9990000000000001])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.947013, [0.08900000000000001, 0.7170000000000001, 0.925, 0.9739999999999999, 0.997, 1.0, 1.0, 1.0, 1.0], [0.003, 0.035, 0.13, 0.21099999999999997, 0.32, 0.458, 0.669, 0.931, 0.992])


# PGD

In [21]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/PGD/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/PGD/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/PGD/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/PGD/0.25098039215686274eps.npz'

In [22]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.982945,
 [0.988, 0.988, 0.988, 0.988, 0.991, 0.992, 0.992, 0.996, 0.998],
 [0.005, 0.039, 0.114, 0.206, 0.3, 0.444, 0.63, 0.912, 0.996])

In [23]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.99, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.004, 0.039, 0.126, 0.225, 0.343, 0.477, 0.657, 0.932, 0.9940000000000001])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.9949999999999999, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.002, 0.039, 0.127, 0.213, 0.315, 0.463, 0.665, 0.927, 0.997])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.9829999999999999, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.01, 0.042, 0.127, 0.22, 0.341, 0.48, 0.666, 0.92, 0.993])


# BIM

In [24]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/BIM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/IMAGENET/ResNet50/BIM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/IMAGENET/ResNet50/BIM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/IMAGENET/ResNet50/BIM/0.25098039215686274eps.npz'

In [25]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.3525445,
 [0.131, 0.161, 0.20200000000000004, 0.222, 0.241, 0.281, 0.354, 0.602, 0.872],
 [0.003, 0.035, 0.11899999999999998, 0.201, 0.303, 0.434, 0.631, 0.92, 0.997])

In [26]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.46281950000000005, [0.368, 0.37200000000000005, 0.389, 0.398, 0.409, 0.434, 0.468, 0.612, 0.861], [0.005, 0.04100000000000001, 0.141, 0.239, 0.34, 0.494, 0.677, 0.929, 0.995])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.586704, [0.537, 0.54, 0.547, 0.551, 0.553, 0.566, 0.571, 0.681, 0.863], [0.003, 0.04100000000000001, 0.117, 0.21600000000000003, 0.317, 0.466, 0.655, 0.929, 0.997])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.615667, [0.587, 0.587, 0.589, 0.59, 0.59, 0.593, 0.603, 0.716, 0.888], [0.004, 0.042, 0.129, 0.223, 0.306, 0.446, 0.66, 0.924, 0.99])


# CW

In [27]:
adv_path1 = 'adv samples/IMAGENET/ResNet50/CW/0.15eps.npz'

In [28]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9521997981836527,
 [0.7144298688193743,
  0.8668012108980827,
  0.9455095862764884,
  0.9646821392532794,
  0.9677093844601413,
  0.9808274470232089,
  0.9889001009081736,
  0.9959636730575177,
  0.9989909182643796],
 [0.008, 0.047, 0.141, 0.23, 0.325, 0.47, 0.645, 0.912, 0.993])